### Phần 1:


### Phần 2:
##### Chuyển dữ liệu host ↔ device:
Host đưa input cho device. Device xử lí trong nội bộ và trả về output cho host. Vậy ta sẽ có các hàm cudaMemcpy để thực hiện các tác vụ:  
+ chuyển input từ host sang device  
+ Khởi tạo các trọng số cho mô hình sẽ được thực hiện trên host rồi chuyển sang device.
+ Chuyển output từ device sang host
+ Chuyển giá trị hàm loss từ device sang host để giám sát quá trình huấn luyện mô hình.

##### Phần tổ chức dữ liệu cho kernel:
Như ta đã biết thì dữ liệu lưu theo index của `threadIdx`, `blockDim`, `blockIdx` chỉ có 3 chiều (x, y, z), trong khi đó, dữ liệu mà ta đang xử lí có tới 4 chiều (H, W, Channel, Batch). Ý tưởng của nhóm em là gộp chiều Channel với Batch lại và lưu và chiều z. Khi đó, một dữ liệu input có chiều (x,y,c,b):
$$\begin{align}
\text{flat\_index} &= (b \cdot C + c) \cdot H \cdot W + W \cdot h + w \\
&= ((b \cdot C + c) \cdot H + h) \cdot W + w
\end{align}$$

##### Tại sao trên các kernel ReLU lại chỉ sử dụng 1 chiều x trong khi các kernel khác thì sử dụng tới 3 chiều ?

##### Tại sao index của chiều batch-channel lại được biểu diễn qua blockIdx.z mà không phải là blockIdx.z * blockDim.z + threadIdx.z ?


##### Tại sao ko cần cudaDeviceSynchronize() sau mỗi kernel call trong forwardPass ?
Vì tất cả job đều nằm trên cùng 1 stream 0 nên các hàm được nằm ở trước chắc chắn sẽ được thực hiện trước.

##### Hiện tại backward pass chưa hợp lí, vì padding scheme của m khác với code gemini


Trong hàm backward, các grad đã được chia trung bình cho toàn bộ pixel trong cả batch. Vậy mà trong hàm updateWeights, grad còn được chia cho batchSize 1 lần nữa. Liệu nó có bị dư ?


##### Trong quá trình làm, liên tục bị gradient explosion nên đã áp dụng gradient clipping cho backprop ổn định hơn